In [5]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [6]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [7]:
#!pip install -U wikipedia

In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [9]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [11]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [14]:
agent= initialize_agent(
    tools, 
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [15]:
agent("What is 25% of 300?")



> Entering new AgentExecutor chain...
Question: What is 25% of 300?
Thought: I need to use a calculator to find the answer.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% * 300"
}
```

Observation: Answer: 75.0
Thought:I have the answer to the question.
Final Answer: 25% of 300 is 75.0.

> Finished chain.


{'input': 'What is 25% of 300?', 'output': '25% of 300 is 75.0.'}

## Wikipedia Example

In [29]:
# question = "Tom M. Mitchell is an American computer scientist \
#and the Founders University Professor at Carnegie Mellon University (CMU)\
#what book did he write?"

question = "Michael S Wolfson is an Google Developer Expert in Android \
and a graduate of Colorado State Universion\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to search for Michael S Wolfson and find out if he has written any books.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Michael S Wolfson"
}
```


Observation: Page: Rancho Mirage, California
Summary: Rancho Mirage is a city in Riverside County, California, United States. The population was 16,999 at the 2020 census, down from 17,218 at the 2010 census, but the seasonal (part-time) population can exceed 20,000. Incorporated in 1973 and located between Cathedral City and Palm Desert, it is one of the nine cities of the Coachella Valley (Palm Springs area).It is home to and has been home to a number of celebrities, including Don Sutton, Frank Sinatra, Dean Martin, Bing Crosby, Sammy Davis Jr., Lucille Ball, Bob Hope, Billie Dove and Gerald and Betty Ford. It is a low-density desert-resort community with resorts, golf courses, and country clubs in the Coachella Valley within the Colorado Desert section of

## Python Agent

In [17]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [18]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [20]:
agent.run(f"""Sort these customers by last name and then first name and print the output: {customer_list}""")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python_REPL
Action Input: 
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted(customers, key=lambda x: (x[1], x[0]))
```
Observation: 
Thought:The output is a sorted list of customers by last name and then first name.
Action: Python_REPL
Action Input: `print(sorted(customers, key=lambda x: (x[1], x[0])))`
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.

"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

### View Detailed Outputs of the chains

In [22]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by last name and then first name and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## Define your Own Tool

In [ ]:
#!pip install DateTime

In [23]:
from langchain.agents import tool
from datetime import date

In [24]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any questions related to knowing todays date. \
    The input should always be an empyt string, and this function will always return todays \
    date - any date mathmatic should occur outside this funtion."""
    return str(date.today())

In [26]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [27]:
try:
    result= agent("what is today's date?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I can use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-08-27
Thought:I have successfully retrieved today's date using the `time` tool.
Final Answer: Today's date is August 27, 2023.

> Finished chain.
